# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_df = pd.read_csv("weather_data.csv")
weather_df

,Unnamed: 0,name,latitude,longitude,humidity,max_temp,cloudiness,wind_speed
0,0,Vila Franca do Campo,37.7167,-25.4333,71,298.38,86,5.15
1,1,George Town,5.4112,100.3354,79,301.12,20,2.06
2,2,Shimoda,34.6667,138.9500,89,299.55,94,2.17
3,3,Hermanus,-34.4187,19.2345,72,293.09,0,3.76
4,4,Rikitea,-23.1203,-134.9692,66,293.81,56,9.66
...,...,...,...,...,...,...,...,...
549,549,Yar-Sale,66.8333,70.8333,50,299.01,97,6.60
550,550,Liyang,29.7057,118.2862,92,299.12,100,1.29
551,551,Kaputa,-8.4689,29.6619,38,302.57,92,4.86
552,552,Alice Springs,-23.7000,133.8833,34,281.90,92,1.54


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
coords = weather_df[["latitude", "longitude"]]
weight = weather_df["humidity"]/300
heatmap = gmaps.heatmap_layer(coords, weights = weight)
fig = gmaps.figure()
fig.add_layer(heatmap)

In [18]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
df1 = weather_df.loc[weather_df['max_temp'] <= 300]
df2 = df1.loc[df1['max_temp'] >= 288]
df3 = df2.loc[df2["wind_speed"] <= 5]
ideal_weather_df = df3.loc[df3["cloudiness"] == 0]

ideal_weather_df

,Unnamed: 0,name,latitude,longitude,humidity,max_temp,cloudiness,wind_speed
3,3,Hermanus,-34.4187,19.2345,72,293.09,0,3.76
33,33,Bredasdorp,-34.5322,20.0403,49,289.97,0,2.53
48,48,Castro,-24.7911,-50.0119,35,297.70,0,3.16
66,66,Port Alfred,-33.5906,26.8910,59,290.85,0,2.81
67,67,Wahpeton,46.2652,-96.6059,77,289.55,0,2.88
74,74,Cape Town,-33.9258,18.4232,65,294.20,0,0.89
83,83,East London,-33.0153,27.9116,56,293.08,0,2.68
122,122,Matagami,49.7502,-77.6328,62,289.68,0,1.61
130,130,Saint George,37.1041,-113.5841,25,296.78,0,0.89
141,141,Kruisfontein,-34.0033,24.7314,41,294.20,0,3.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = None
hotel_df

<ipython-input-48-1c29596d35a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = None


,Unnamed: 0,name,latitude,longitude,humidity,max_temp,cloudiness,wind_speed,Hotel Name
3,3,Hermanus,-34.4187,19.2345,72,293.09,0,3.76,None
33,33,Bredasdorp,-34.5322,20.0403,49,289.97,0,2.53,None
48,48,Castro,-24.7911,-50.0119,35,297.70,0,3.16,None
66,66,Port Alfred,-33.5906,26.8910,59,290.85,0,2.81,None
67,67,Wahpeton,46.2652,-96.6059,77,289.55,0,2.88,None
74,74,Cape Town,-33.9258,18.4232,65,294.20,0,0.89,None
83,83,East London,-33.0153,27.9116,56,293.08,0,2.68,None
122,122,Matagami,49.7502,-77.6328,62,289.68,0,1.61,None
130,130,Saint George,37.1041,-113.5841,25,296.78,0,0.89,None
141,141,Kruisfontein,-34.0033,24.7314,41,294.20,0,3.74,None


In [53]:
params = {"radius":5000, "types":"lodging", "key":g_key}

for index, row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = str(row["latitude"]) + "," + str(row["longitude"])
    data = requests.get(url, params = params).json()
    
    hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    
hotel_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,name,latitude,longitude,humidity,max_temp,cloudiness,wind_speed,Hotel Name
3,3,Hermanus,-34.4187,19.2345,72,293.09,0,3.76,Misty Waves Boutique Hotel
33,33,Bredasdorp,-34.5322,20.0403,49,289.97,0,2.53,Bredasdorp Country Manor
48,48,Castro,-24.7911,-50.0119,35,297.70,0,3.16,CHACARA BAILLY
66,66,Port Alfred,-33.5906,26.8910,59,290.85,0,2.81,The Halyards Hotel
67,67,Wahpeton,46.2652,-96.6059,77,289.55,0,2.88,Select Inn Breckenridge
74,74,Cape Town,-33.9258,18.4232,65,294.20,0,0.89,Southern Sun Waterfront Cape Town
83,83,East London,-33.0153,27.9116,56,293.08,0,2.68,Tu Casa
122,122,Matagami,49.7502,-77.6328,62,289.68,0,1.61,Hôtel Matagami
130,130,Saint George,37.1041,-113.5841,25,296.78,0,0.89,Best Western Plus Abbey Inn
141,141,Kruisfontein,-34.0033,24.7314,41,294.20,0,3.74,Oyster Bay House Rental


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [56]:
# Add marker layer ontop of heat map


marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))